In [46]:
#Importing libraries

import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#import Random forest classifiers
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns


In [47]:
import pandas as pd

In [61]:
labels = pd.read_csv("Galaxy_data/training_solutions_rev1.csv")
labels.shape

(61578, 38)

In [62]:
pd.set_option('display.max_columns',None)
labels=labels.drop(labels[labels['Class1.3'] > 0.5].index)

In [63]:
labels = labels[['GalaxyID', 'Class1.1', 'Class1.2', 'Class6.1']]
labels['Result'] = 'i'
labels.loc[labels['Class1.1'] > 0.6, 'Result'] = 'e'
labels.loc[labels['Class1.2'] > 0.6, 'Result'] = 's'
labels.loc[labels['Class6.1'] > 0.6, 'Result'] = 'o'



In [64]:
labels.head()

,GalaxyID,Class1.1,Class1.2,Class6.1,Result
0,100008,0.383147,0.616853,0.054453,s
1,100023,0.327001,0.663777,0.189149,s
2,100053,0.765717,0.177352,0.000000,e
3,100078,0.693377,0.238564,0.320398,e
4,100090,0.933839,0.000000,0.029383,e


In [67]:
elip_df = labels[labels['Result'] == 'e']
spiral_df = labels[labels['Result'] == 's']
odd_df = labels[labels['Result'] == 'o']

# Sample 5000 values from each category
category1_sampled = elip_df.sample(n=5000, random_state=42)
category2_sampled = spiral_df.sample(n=5000, random_state=42)
category3_sampled = odd_df.sample(n=5000, random_state=42)

# Concatenate the sampled DataFrames back together
sampled_df = pd.concat([category1_sampled, category2_sampled, category3_sampled])

galaxy =  sampled_df.sort_values(by='GalaxyID')

print(galaxy)


       GalaxyID  Class1.1  Class1.2  Class6.1 Result
2        100053  0.765717  0.177352  0.000000      e
5        100122  0.738832  0.238159  0.197930      e
6        100123  0.462492  0.456033  0.687647      o
8        100134  0.021834  0.976952  0.611499      o
16       100263  0.179654  0.818530  0.913055      o
...         ...       ...       ...       ...    ...
61569    999890  0.308029  0.691971  0.086050      s
61570    999893  0.410999  0.519560  0.801448      o
61574    999950  0.901216  0.098784  0.127644      e
61575    999958  0.202841  0.777376  0.839186      o
61576    999964  0.091000  0.909000  0.864000      o

[15000 rows x 5 columns]


In [45]:
# Filter rows where 'Result' is 'o'
result_o = labels[labels['Result'] == 'e']

# Display the resulting DataFrame
print(result_o)


       GalaxyID  Class1.1  Class1.2  Class6.1 Result
4        100090  0.933839  0.000000  0.029383      e
32       100479  0.841554  0.158446  0.205023      e
41       100672  0.931725  0.036703  0.201979      e
43       100724  0.862362  0.137638  0.000000      e
44       100727  0.816171  0.156441  0.042799      e
...         ...       ...       ...       ...    ...
61556    999655  0.811430  0.188570  0.022833      e
61557    999680  0.836536  0.148022  0.058969      e
61564    999781  0.848000  0.064000  0.047000      e
61565    999786  0.800415  0.165614  0.086267      e
61574    999950  0.901216  0.098784  0.127644      e

[8080 rows x 5 columns]


In [42]:
labels=labels.drop(labels[labels['Result'] == 'i'].index)
print(labels)

       GalaxyID  Class1.1  Class1.2  Class6.1 Result
4        100090  0.933839  0.000000  0.029383      e
8        100134  0.021834  0.976952  0.611499      s
14       100237  0.164391  0.800759  0.132726      s
16       100263  0.179654  0.818530  0.913055      o
19       100322  0.091987  0.908013  0.000000      s
...         ...       ...       ...       ...    ...
61568    999875  0.020285  0.979715  0.071272      s
61570    999893  0.410999  0.519560  0.801448      o
61574    999950  0.901216  0.098784  0.127644      e
61575    999958  0.202841  0.777376  0.839186      o
61576    999964  0.091000  0.909000  0.864000      o

[25562 rows x 5 columns]


In [43]:
galaxy_ids = labels['GalaxyID'].to_numpy()

# Display the array of GalaxyID
print(galaxy_ids)

[100090 100134 100237 ... 999950 999958 999964]


In [ ]:
from PIL import Image, ImageOps
from numpy import asarray
import os

folder_path = "Galaxy_data/images_training_rev1/"
output_dict = {}

for galaxy_id in galaxy_ids:
    filename = f"{galaxy_id}.jpg"
    filepath = os.path.join(folder_path, filename)

    if os.path.exists(filepath):
        # load the image and convert into numpy array
        img = Image.open(filepath)
        img_gray = ImageOps.grayscale(img)
        numpydata = asarray(img_gray).flatten()

        # store the numpy array in the dictionary with the galaxy_id as the key
        output_dict[galaxy_id] = numpydata

# Now output_dict contains numpy arrays for all specified galaxy IDs


In [ ]:
import numpy as np
x=np.array(list(output_dict.values())).mean()

In [ ]:
#import csv


# Specify the CSV file path
#csv_file_path = 'galaxy.csv'

# Writing the data to the CSV file
#with open(csv_file_path, 'w', newline='') as csv_file:
#    csv_writer = csv.writer(csv_file)

    # Write the header row with keys as column names
#    header = ['GalaxyID'] + [f'Pixel_{i}' for i in range(len(output_dict[100008].ravel()))]
#    csv_writer.writerow(header)

    # Write data rows
#    for key, value in output_dict.items():
        # Flatten the array to a 1D list for the row
#        row_data = [key] + value.ravel().tolist()
#        csv_writer.writerow(row_data)

#print(f'CSV file has been created at: {csv_file_path}')


In [ ]:
output_dict[100008]

In [ ]:
len(output_dict[100008].ravel())

424.0